# Problem Statement

Given a list of integer numbers, look for a negative number in the list. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of number in case

# Solution strategy

Encode the list in the qubits. Total number of qubits are equal to the number of elements in the list. Encoder is an oracle or a black box. Encoder will flip the state of qubit to |1> if the number is positive and leave it in |0> state if the number is negative, corresponding to the index of element in the list. Then we use multi control toffoli to flip the auxilary qubit to |1> if all the input elements are positive. If the output state is |0>, it implies that the input has one or more negative element

In [1]:
# Required modules
import numpy as np

from qiskit import IBMQ,Aer,transpile, execute
from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.visualization import plot_histogram

print('Import Done')


Import Done


In [2]:
integer = [2,3,-8,10]

In [3]:
backend = Aer.get_backend('qasm_simulator') # Using qasm simulator, other backends could be used as well

In [4]:
def oracle(list):
    '''Function takes a list and flip the qubits based on the
    sign of corresponding element in the list
    Return:
        return qiskit circuit
    '''
    qc = QuantumCircuit(len(list), name= 'Oracle')
    for i, int in enumerate(list):
        if int>=0:
            qc.x(i)
    return qc

In [5]:
def find_negative_numbers(x):
    """x: list to be searched"""
    if type(x) == list:          
        fn = QuantumCircuit(len(x)+1,1) # Quantum register= len(list) +1, classical register=1
        # Adding oracle circuit
        fn.compose(oracle(x),list(range(len(x))),inplace= True)
        # Multi control toffoli flips the state of measurement qubit based on encoded qubits
        fn.mct(list(range(len(x))),fn.qregs[0][-1])        
        # Measuruing the measurement qubit
        fn.measure(fn.qregs[0][-1],0)
        counts = backend.run(transpile(fn,backend)).result().get_counts(fn)
        # max_key = 0 means True or else False
        max_key = max(counts, key=lambda key: counts[key])
        return bool(max_key=='0')
    else:
        raise TypeError("Input must be a list")
       
     

In [6]:
A = find_negative_numbers(integer)

In [7]:
print(A)

True
